In [78]:
!pip install nose

In [79]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nose.tools as nt

In [80]:
!wget -c 'https://drive.google.com/uc?export=download&id=1rh6Rgjoyloyp2hPhTlOBD16yc7sc3gPy' -O data.zip
!unzip -o data.zip

--2023-11-29 08:32:10--  https://drive.google.com/uc?export=download&id=1rh6Rgjoyloyp2hPhTlOBD16yc7sc3gPy
Resolving drive.google.com (drive.google.com)... 142.251.167.138, 142.251.167.113, 142.251.167.139, ...
Connecting to drive.google.com (drive.google.com)|142.251.167.138|:443... connected.
HTTP request sent, awaiting response... 416 Requested range not satisfiable

    The file is already fully retrieved; nothing to do.

Archive:  data.zip
  inflating: BLOSUM62.txt            
  inflating: Q2b_reference.txt       


In [81]:
!ls *.txt

BLOSUM62.txt  Q2b_reference.txt


# Needleman-Wunsch Algorithm

In [82]:
def read_blosum62(path):
    delta = {}
    with open(path, 'r') as f:
        lines = f.readlines()[6:]
        keys = lines[0].split()
        keys[-1] = '-'
        for i, line in enumerate(lines[1:]):
            delta[keys[i]] = {k : int(v) for (k,v) in zip(keys, line.split()[1:])}
    return delta

blosum = read_blosum62('./BLOSUM62.txt')

UP = (-1,0)
LEFT = (0, -1)
TOPLEFT = (-1, -1)
ORIGIN = (0, 0)

def traceback_global(v, w, pointers):
    i,j = len(v), len(w)
    new_v = []
    new_w = []
    while True:
        di, dj = pointers[i][j]
        if (di,dj) == LEFT:
            new_v.append('-')
            new_w.append(w[j-1])
        elif (di,dj) == UP:
            new_v.append(v[i-1])
            new_w.append('-')
        elif (di,dj) == TOPLEFT:
            new_v.append(v[i-1])
            new_w.append(w[j-1])
        i, j = i + di, j + dj
        if (i <= 0 and j <= 0):
            break
    return ''.join(new_v[::-1])+'\n'+''.join(new_w[::-1])

def needleman_wunsch(v, w, delta):
    M = [[0 for j in range(len(w)+1)] for i in range(len(v)+1)]
    pointers = [[ORIGIN for j in range(len(w)+1)] for i in range(len(v)+1)]
    score, alignment = None, None

    for i in range(1, len(v) + 1):
      M[i][0] = M[i - 1][0] + delta[v[i - 1]]['-']
      pointers[i][0] = UP

    for j in range(1, len(w) + 1):
      M[0][j] = M[0][j - 1] + delta['-'][w[j - 1]]
      pointers[0][j] = LEFT

    for i in range(1, len(v) + 1):
      for j in range(1, len(w) + 1):
        scores = [M[i - 1][j - 1] + delta[v[i - 1]][w[j - 1]], M[i - 1][j] + delta[v[i - 1]]['-'], M[i][j - 1] + delta['-'][w[j - 1]]]
        M[i][j] = max(scores)
        if M[i][j] == scores[0]:
          pointers[i][j] = TOPLEFT
        elif M[i][j] == scores[1]:
          pointers[i][j] = UP
        else:
          pointers[i][j] = LEFT
    score = M[len(v)][len(w)]
    # --------------------------------------------
    alignment = traceback_global(v,w, pointers)
    return score, alignment

keys = ['A', 'C', 'T', 'G', '-']
delta = {}
for i in range(len(keys)):
    delta[keys[i]] = {k : v for (k,v) in zip(keys, [1 if keys[i] == keys[j]  else -1 for j in range(len(keys))])}

needleman_wunsch("TAGATA", "GTAGGCTTAAGGTTA", delta)

(-3, '-TA-G----A---TA\nGTAGGCTTAAGGTTA')

# Hirschberg's Algorithm

In [82]:
def hirschberg(seq1, seq2):
    # Your Hirschberg implementation here
    pass

def hirschberg_recursive(seq1, seq2):
    # Helper function for recursion
    pass

# Visualization